# clvlib Tutorial: Lyapunov Analysis with NumPy

This notebook demonstrates how to compute Lyapunov exponents, backward/covariant Lyapunov vectors,
and instantaneous covariant Lyapunov exponents (ICLEs) for the Lorenz '63 system using `clvlib.numpy`.


## Prerequisites

Ensure `clvlib` is installed in your environment (see the project README for installation instructions).
This walkthrough relies only on NumPy, SciPy, and Matplotlib.


In [ ]:
import numpy as np
import matplotlib.pyplot as plt

from clvlib.numpy import (
    lyap_analysis_from_ic,
    compute_angles,
    principal_angles,
    compute_ICLE,
)

## Define the Lorenz '63 system

We work with the classical chaotic Lorenz attractor and its analytical Jacobian.


In [ ]:
SIGMA = 10.0
RHO = 28.0
BETA = 8.0 / 3.0


def lorenz(t: float, x: np.ndarray) -> np.ndarray:
    return np.array(
        [
            SIGMA * (x[1] - x[0]),
            x[0] * (RHO - x[2]) - x[1],
            x[0] * x[1] - BETA * x[2],
        ],
        dtype=float,
    )


def jacobian(t: float, x: np.ndarray) -> np.ndarray:
    return np.array(
        [
            [-SIGMA, SIGMA, 0.0],
            [RHO - x[2], -1.0, -x[0]],
            [x[1], x[0], -BETA],
        ],
        dtype=float,
    )

## Integrate state and variational equations

`lyap_analysis_from_ic` integrates the trajectory and variational system simultaneously, returning
Lyapunov exponent estimates, backward Lyapunov vectors (BLVs), CLVs, and the trajectory samples.


In [ ]:
times = np.linspace(0.0, 40.0, 4001)  # 4000 integration steps
x0 = np.array([8.0, 0.0, 30.0], dtype=float)

LE, LE_history, BLV_history, CLV_history, trajectory = lyap_analysis_from_ic(
    lorenz,
    jacobian,
    x0,
    times,
    stepper="rk4",
    qr_method="householder",
    ginelli_method="ginelli",
)

print("Asymptotic Lyapunov exponents:", LE)

## Visualise the convergence of Lyapunov exponents

Plot each exponent as a function of time to check convergence behaviour.


In [ ]:
plt.figure(figsize=(8, 4))
for k in range(LE_history.shape[1]):
    plt.plot(times, LE_history[:, k], label=f"LE_{k + 1}")
plt.axhline(0.0, color="k", linewidth=0.8, linestyle="--")
plt.xlabel("Time")
plt.ylabel("Lyapunov exponent estimate")
plt.title("Lyapunov exponent convergence (Lorenz 63)")
plt.legend()
plt.tight_layout()
plt.show()

## Analyse CLVs and instantaneous exponents

Compare backward and covariant Lyapunov vectors at the final sample, compute principal angles
between subspaces, and derive instantaneous CLVs with a chosen sampling stride.


In [ ]:
cosine, theta = compute_angles(BLV_history[-1], CLV_history[-1])
print("Angles between final BLVs and CLVs (radians):", theta)

angles = principal_angles(BLV_history[:, :, :2], CLV_history[:, :, :2])
print("Principal angles shape:", angles.shape)

icle = compute_ICLE(jacobian, trajectory, times, CLV_history, k_step=10)
print("ICLE shape:", icle.shape)